In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import pandas as pd
import numpy as np
import pyBigWig
import math
k562_positive=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/K562_unt.sort.bed.gz_plus.bw')
k562_negative=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/K562_unt.sort.bed.gz_minus.bw')
h3k27ac_bigwig=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k27acStdSig.bigWig')
h3k27me3_bigwig=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k27me3StdSig.bigWig')
h3k4me3_bigwig=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k4me3StdSig.bigWig')
h3k4me1_bigwig=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k4me1StdSig.bigWig')
h3k27ac_peak_replicated_bed=pd.read_csv('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k27acStdPk.broadPeak',sep='\t',header=None)
h3k27ac_peak_replicated_bed['diff']=h3k27ac_peak_replicated_bed[2]-h3k27ac_peak_replicated_bed[1]
h3k27ac_peak_replicated_bed['absolute_diff']=3000-h3k27ac_peak_replicated_bed['diff']
h3k27ac_peak_replicated_bed['left']=h3k27ac_peak_replicated_bed['absolute_diff'].apply(lambda x : math.floor(float(x)/2))
h3k27ac_peak_replicated_bed['right']=h3k27ac_peak_replicated_bed['absolute_diff'].apply(lambda x : math.ceil(float(x)/2))
h3k27ac_peak_replicated_bed['left_new']=h3k27ac_peak_replicated_bed[1]-h3k27ac_peak_replicated_bed['left']
h3k27ac_peak_replicated_bed['right_new']=h3k27ac_peak_replicated_bed[2]+h3k27ac_peak_replicated_bed['right']
h3k27ac_peak_replicated_bed['new_diff']=h3k27ac_peak_replicated_bed['right_new']-h3k27ac_peak_replicated_bed['left_new']
h3k27ac_peak_replicated_bed['left_new']=h3k27ac_peak_replicated_bed['left_new'].astype(int)
h3k27ac_peak_replicated_bed['right_new']=h3k27ac_peak_replicated_bed['right_new'].astype(int)
#h3k27ac_peak_replicated_bed[[0,'left_new','right_new']].to_csv('/oak/stanford/groups/akundaje/laks/proseq_chromputer/h3k27ac_peak_list.bed',sep='\t',header=None,index=False)
#os.system('bedtools makewindows -g /oak/stanford/groups/akundaje/laks/proseq_chromputer/hg19.chrom.sizes -w 2000 -s 200 > /oak/stanford/groups/akundaje/laks/proseq_chromputer/genome.2kb.bed')
#os.system('bedtools subtract -a /oak/stanford/groups/akundaje/laks/proseq_chromputer/genome.2kb.bed -b /oak/stanford/groups/akundaje/laks/proseq_chromputer/h3k27ac_peak_list.bed > /oak/stanford/groups/akundaje/laks/proseq_chromputer/genome.2kb_h3k27ac_k562_hg19_negatives.bed')



#
total_sum=0
for i in range(0,22):
	if i==0:
		chrm='chrX'
	else:
		chrm='chr'+str(i)
	length=k562_positive.chroms(chrm)
	values=k562_positive.values(chrm,0,length)
	a=np.array(values)
	a[np.isnan(a)] = 0
	total_sum=total_sum+np.sum(a)

total_sum_negative=0
for i in range(0,22):
	if i==0:
		chrm='chrX'
	else:
		chrm='chr'+str(i)
	length=k562_negative.chroms(chrm)
	values=k562_negative.values(chrm,0,length)
	a=np.array(values)
	a[np.isnan(a)] = 0
	total_sum_negative=total_sum_negative+np.sum(a)

In [2]:
#preparing the positive peak file
h3k27ac_positives=h3k27ac_peak_replicated_bed[[0,'left_new','right_new']]
h3k27ac_positives.columns=['chr','start','end']
#
h3k27ac_positives_validation=h3k27ac_positives[h3k27ac_positives['chr']=='chr22']
h3k27ac_positives_testing=h3k27ac_positives[h3k27ac_positives['chr']=='chr19']
#
h3k27ac_positives=h3k27ac_positives[h3k27ac_positives['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr20','chr21','chrX'])]
h3k27ac_positives['id']=h3k27ac_positives['chr'].astype(str)+'_'+h3k27ac_positives['start'].astype(str)+'_'+h3k27ac_positives['end'].astype(str)
#
#h3k27ac_negatives=pd.read_csv('/oak/stanford/groups/akundaje/laks/proseq_chromputer/genome.2kb_h3k27ac_k562_hg19_negatives.bed',sep='\t',header=None)
#h3k27ac_negatives=h3k27ac_negatives[h3k27ac_negatives[1]>20000]
#h3k27ac_negatives.columns=['chr','start','end']
#h3k27ac_negatives=h3k27ac_negatives[h3k27ac_negatives['end']-h3k27ac_negatives['start']==2000]
#
#h3k27ac_negatives_validation=h3k27ac_negatives[h3k27ac_negatives['chr']=='chr22']
#
#h3k27ac_negatives['id']=h3k27ac_negatives['chr'].astype(str)+'_'+h3k27ac_negatives['start'].astype(str)+'_'+h3k27ac_negatives['end'].astype(str)
#h3k27ac_negatives=h3k27ac_negatives[h3k27ac_negatives['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chrX'])]
#
#validation_data=h3k27ac_positives_validation.append(h3k27ac_negatives_validation)
#print validation_data[0:5]
validation_data=h3k27ac_positives_validation
test_data=h3k27ac_positives_testing

In [ ]:
input_width=13000 
input_dimension=1
number_of_convolutions=16
filters=[32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32]
filter_dim=[11,11,11,11,11,11,11,11,21,21,21,21,41,41,41,41]
dilation=[1,1,1,1,4,4,4,4,10,10,10,10,25,25,25,25]
activations='relu'
bn_true=True


def BatchNormalization_mod(conv, bn_flag=True):
    from keras.layers.normalization import BatchNormalization
    if bn_flag:
        return BatchNormalization()(conv)
    else:
        return conv


def res_block(conv,num_filter,f_width,act,d_rate,i,bn_true=True):
    import tensorflow as tf
    import keras
    from keras import backend as K
    from keras.layers.pooling import GlobalMaxPooling1D,MaxPooling2D,MaxPooling1D
    from keras.models import Sequential,Model
    from keras.layers import Dense,Activation,Dropout,Flatten,Reshape,Input, Embedding, LSTM, Dense,Concatenate
    from keras.layers.convolutional import Conv1D,Conv2D,Cropping1D
    from keras.layers.normalization import BatchNormalization
    from keras.regularizers import l1,l2
    from keras.optimizers import SGD,RMSprop,Adam
    from sklearn.metrics import average_precision_score
    crop_id=Cropping1D(d_rate*(f_width-1))(conv)
    conv1 = BatchNormalization_mod(conv,bn_true)
    conv1 = Activation("relu")(conv1)
    conv1 = Conv1D(num_filter,f_width,dilation_rate=d_rate,padding="valid",name='conv_'+str(i)+'_a')(conv1)
    conv1 = BatchNormalization_mod(conv1,bn_true)
    conv1 = Activation("relu")(conv1)
    conv1 = Conv1D(num_filter,f_width,dilation_rate=d_rate,padding="valid",name='conv_'+str(i)+'_b')(conv1)
    return keras.layers.Add()([conv1, crop_id])

def build1d_model_residual(input_width,input_dimension,number_of_convolutions,filters,filter_dim,dilation,activations,bn_true=True,max_flag=True):
    import tensorflow as tf
    import keras
    from keras import backend as K
    from keras.layers.pooling import GlobalMaxPooling1D,MaxPooling2D,MaxPooling1D
    from keras.models import Sequential,Model
    from keras.layers import Dense,Activation,Dropout,Flatten,Reshape,Input, Embedding, LSTM, Dense,Concatenate
    from keras.layers.convolutional import Conv1D,Conv2D
    from keras.layers.normalization import BatchNormalization
    from keras.regularizers import l1,l2
    from keras.optimizers import SGD,RMSprop,Adam
    from sklearn.metrics import average_precision_score
    input1=Input(shape=(13000,1), name='proseq_positive')
    input2=Input(shape=(13000,1), name='proseq_negative')
    inputlayer=keras.layers.add()([input1, input2])
    conv=Conv1D(32,1, padding='same',activation='relu',name = 'upsampling')(inputlayer)
    for i in range(0,number_of_convolutions):
            conv = res_block(conv,filters[i],filter_dim[i],activations,dilation[i],i,bn_true)
    conv= Conv1D(32, 1,padding='valid', activation='relu',name='down_sampling')(conv)
    output=Conv1D(1,1,activation='relu',name='output')(conv)
    model = Model(input=[input1,input2],output=[output])
    model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    return model



def validation_loss(model,validation_data):
    validation_mse=[]
    for i in validation_data.values:
            positive_values=np.nan_to_num(k562_positive.values(i[0],i[1]-5000,i[2]+5000))/(total_sum+np.abs(total_sum_negative))
            negative_values=np.nan_to_num(k562_negative.values(i[0],i[1]-5000,i[2]+5000))/(total_sum+np.abs(total_sum_negative))
            h3k27ac_value=np.arcsinh(np.nan_to_num(h3k27ac_bigwig.values(i[0],i[1],i[2])))
            loss_validation=model.evaluate({'proseq_positive':np.expand_dims(np.expand_dims(np.asarray(positive_values),1),0),\
                'proseq_negative':np.expand_dims(np.expand_dims(np.asarray(negative_values),1),0) },\
                {'output':np.expand_dims(np.expand_dims(np.nan_to_num(np.asarray(h3k27ac_value)),1),0)},\
                batch_size=1,verbose=0)
            validation_mse.append(loss_validation[0])
    return float(sum(validation_mse))/len(validation_mse)


def calculate_input(X_train):
    xtrain_positive=[]
    xtrain_negative=[]
    ytrain=[]
    for i in X_train.values:
        try:
        	xtrain_positive.append(np.nan_to_num(k562_positive.values(i[0],int(i[1])-5000,int(i[2])+5000))/(total_sum+np.abs(total_sum_negative) ))
        	xtrain_negative.append(np.nan_to_num(k562_negative.values(i[0],int(i[1])-5000,int(i[2])+5000))/(total_sum+np.abs(total_sum_negative)))
        	ytrain.append(np.arcsinh(np.nan_to_num(h3k27ac_bigwig.values(i[0],int(i[1]),int(i[2])))))
    	except Exception as e:
    		print str(e)
    		print i
    xtrain_positive=np.expand_dims(np.asarray(xtrain_positive),2)
    xtrain_negative=np.expand_dims(np.asarray(xtrain_negative),2)
    ytrain=np.expand_dims(np.asarray(ytrain),2)
    return (xtrain_positive,xtrain_negative,ytrain)


model=build1d_model_residual(input_width,input_dimension,number_of_convolutions,filters,filter_dim,dilation,activations,bn_true)
validation_loss_val=10000000
patience=0
while True:
    used_positives=pd.Series()
    #used_negatives=pd.Series()
    flag=0
    while True:
        try:
            peaks_regions=h3k27ac_positives[~(h3k27ac_positives['id'].isin(used_positives))].sample(n=150,random_state=95918)
        except:
            used_positives=pd.Series()
            print used_positives.shape
            #print used_negatives.shape
            flag=flag+1
            if flag ==1:
            	break
            #peaks_regions=h3k27ac_positives[~(h3k27ac_positives['id'].isin(used_positives))].sample(n=150,random_state=95918)
            print '#########################breaking#################'
            break
        #try:
        #    nonpeaks_regions=h3k27ac_negatives[~(h3k27ac_negatives['id'].isin(used_negatives))].sample(n=150,random_state=95918)
        #except:
        #    used_negatives=pd.Series()
        used_positives=used_positives.append(peaks_regions['id'])
        #used_negatives=used_negatives.append(nonpeaks_regions['id'])
        #print used_positives.shape
        #print used_negatives.shape
        #X_train=peaks_regions.append(nonpeaks_regions)
        X_train=peaks_regions
        X_train=X_train.sample(frac=1)
        X_train_positive,X_train_negative,Y_train_histone=calculate_input(X_train)
        a=model.fit({'proseq_positive': X_train_positive , 'proseq_negative': X_train_negative},{'output':Y_train_histone},epochs=1, batch_size=50,verbose=0)    
    new_validation_loss_val=validation_loss(model,validation_data)
    if new_validation_loss_val < validation_loss_val:
        print '#############change in value ##############'
        print new_validation_loss_val
        print validation_loss_val
        print '#############change in value ##############'
        validation_loss_val=new_validation_loss_val
        model.save_weights('/oak/stanford/groups/akundaje/laks/proseq_chromputer/13kb_3kb_h3k27ac_ADD.hdf5')
    else:
    	print '############# no change in value ##############'
        print new_validation_loss_val
        print validation_loss_val
        print '############# no change in value ##############'
    	model.load_weights('/oak/stanford/groups/akundaje/laks/proseq_chromputer/13kb_3kb_h3k27ac_ADD.hdf5')
        patience=patience+1
    if patience==5:
        break